In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import torch
from inpainting.inpainters.rgb_misgan import RGBMisganInpainterInterface
from inpainting.datasets.celeba import train_val_datasets, DEFAULT_MASK_CONFIGS
from pathlib import Path
from inpainting.datasets.utils import RandomRectangleMaskConfig
from inpainting.datasets import mask_coding as mc
from torchvision.datasets import MNIST, FashionMNIST
from torch.utils.data import DataLoader, TensorDataset
import pickle
from tqdm import tqdm
import numpy as np
from inpainting.visualizations import samples as vis
from inpainting.visualizations.digits import  rgb_with_mask
import matplotlib.pyplot as plt
from inpainting.utils import predictions_for_entire_loader
from inpainting.datasets.mask_coding import KNOWN, UNKNOWN_LOSS

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
experiment_path = Path("../results/celeba/misgan_64x64")

In [ ]:
inpainter = RGBMisganInpainterInterface()
chckp = torch.load(
    "/home/mprzewiezlikowski/uj/misgan/results/celeba/"
    "impute_0313.095138_tau_0.5_maskgen_fusion_coef_0.1_0.1_0_block_32/log/checkpoint.pth")
# inpainter.imputer.load_state_dict(chckp)
chckp.keys()

In [ ]:
inpainter.imputer.load_state_dict(chckp["imputer"])

In [ ]:
ds_train, ds_val = train_val_datasets(
    Path.home() / "uj/.data/", 
    mask_configs=[RandomRectangleMaskConfig(UNKNOWN_LOSS, 32, 32)],
    resize_size=(100, 100),
    crop_size=(64,64)
)

fig, axes = plt.subplots(10, 10, figsize=(15, 15))
for i in range(100):
    (x,j), y = ds_train[i]
#     print(x.shape)
    ax = axes[i // 10, i%10]
#     ax.set_title(f"{y}")
    rgb_with_mask(x.numpy(), j.numpy(), ax)
train_fig = plt.gcf()
train_fig.savefig(experiment_path / "train.png")
plt.show()

len(ds_train), len(ds_val)

In [ ]:
batch_size=256
dl_train = DataLoader(ds_train, batch_size, shuffle=True)
dl_val = DataLoader(ds_val, batch_size, shuffle=True)

In [ ]:
for (x,j),y in dl_val:
    break

x.shape

In [ ]:
results = predictions_for_entire_loader(
    inpainter,
    dl_val,
    device
)

len(results)

In [ ]:
results_path = experiment_path
results_path.mkdir(parents=True, exist_ok=True)

with (results_path / "val_predictions.pkl").open("wb") as f:
    pickle.dump(results, f)

In [ ]:
[
    t.shape for t in results[0]
]

In [ ]:
fig, ax = plt.subplots(10, 11, figsize=(15,15)
)

for i in range(10):
    vis.visualize_sample(
        *results[i],
        drawing_fn=rgb_with_mask,
        title_prefixes=dict(),
        ax_row=ax[i]
    )